In [ ]:
from sklearn import datasets
import numpy as np

# Cargamos el conjunto de datos Iris
iris = datasets.load_iris()
X = iris.data
y = iris.target


# Seleccionamos solo las clases Iris Setosa (0) e Iris Virginica (2) y reemplazamos las clases por -1 y 1 para ajustarse al problema SVM
indices = np.where((y == 0) | (y == 2))
X = X[indices]
y = y[indices]
y = np.where(y == 0, -1, 1)


Hemos cargado con éxito el conjunto de datos de Iris y seleccionado solo las clases Iris Setosa e Iris Virginica. Las características (X) tienen dimensiones (100, 4), lo que significa que hay 100 muestras con 4 características cada una. Las etiquetas (y) son un vector de 100 elementos, que ahora contienen -1 para Iris Setosa y 1 para Iris Virginica.

El siguiente paso es construir el problema dual de SVM. Para ello, definiremos la función objetivo y las restricciones del problema. Luego, utilizaremos la función de kernel radial y resolveremos el problema de optimización con el método de puntos interiores utilizando la biblioteca cvxopt.


**Parámetros del Problema de Optimización Cuadrática**
El problema de optimización para el SVM dual:
 $\text{Maximizar:}
\quad ∑_{i=1}^{n}\alpha_i-\frac{1}{2}∑_{i,j=1}^{n} y_iy_jα_iα_jx_i⋅x_j
\\
\text{Sujeto a:}
\quad 0 \leq α_i \leq C,
\quad ∑_{i,j=1}^{n}α_iy_i =0
$

 se puede escribir en forma matricial como:
  $\text{Minimizar:}
\quad \frac{1}{2} \alpha^T P \alpha - q^T \alpha
\\
\text{Sujeto a:}
\quad G \alpha \leq h ,
\quad A \alpha = b
$

Donde:


*   α son los multiplicadores de Lagrange.
*   P es una matriz que en el caso de SVM es el producto del kernel de todas las muestras y el producto externo de las etiquetas de las clases, es decir, $P = y_iy_jK_{ij}$
*  q es un vector columna con todos los elementos igual a -1, ya que en el problema dual, buscamos maximizar la suma de los $α_i$ (minimizar el negativo de la suma)
*   G y h son matrices que representan las restricciones de desigualdad de los $α_i$, que deben estar entre 0 y el valor de penalizacion C. Aquí, G es una matriz que se construye para mantener $α\geq 0$ y $α\leq C$    
*   A y b representan la restricción de igualdad que asegura que la suma de los productos de los multiplicadores de Lagrange y las etiquetas de las clases es 0, es decir, $∑y_iα_i=0$












In [ ]:
from cvxopt import matrix, solvers

# Definimos la función del kernel radial (RBF)
def kernel_rbf(x, y, gamma=0.5):
    return np.exp(-gamma * np.linalg.norm(x - y) ** 2)

# Construimos la matriz del kernel para todas las muestras
K = np.zeros((len(X), len(X)))
for i in range(len(X)):
    for j in range(len(X)):
        K[i,j] = kernel_rbf(X[i], X[j])

# Configuramos los parámetros para el problema de optimización cuadrática
P = matrix(np.outer(y, y) * K)
q = matrix(-np.ones((len(X), 1)))
G = matrix(np.vstack((-np.eye(len(X)), np.eye(len(X)))))
h = matrix(np.hstack((np.zeros(len(X)), np.ones(len(X)) * 1))) # C=1
A = matrix(y.reshape(1, -1).astype(np.double))
b = matrix(np.zeros(1), (1, 1))

# Resolvemos el problema de optimización cuadrática
solvers.options['show_progress'] = True
solvers.options['abstol'] = 1e-10
solvers.options['reltol'] = 1e-10
solvers.options['feastol'] = 1e-10

# Medimos el tiempo antes de comenzar la optimización
import time
start_time = time.time()

solution = solvers.qp(P, q, G, h, A, b)

# Tiempo total de ejecución y número de iteraciones
execution_time = time.time() - start_time
num_iterations = solution['iterations']

# Mostramos el tiempo de ejecución y el número de iteraciones
execution_time, num_iterations

# Extrae los multiplicadores de Lagrange
lambdas = np.array(solution['x']).flatten()

# Identifica los vectores de soporte (aquellos con lambda no insignificante)
support_vectors_idx = lambdas > 1e-5
support_vectors = X[support_vectors_idx]
support_vector_labels = y[support_vectors_idx]
support_vector_lambdas = lambdas[support_vectors_idx]

# Muestra la información relevante
print("Número de vectores de soporte:", support_vectors.shape[0])
print("Vectores de soporte:")
print(support_vectors)


     pcost       dcost       gap    pres   dres
 0: -1.2253e+00 -1.1560e+02  4e+02  1e+00  6e-16
 1:  7.6781e-01 -4.0758e+01  4e+01  3e-16  7e-16
 2: -1.8193e+00 -5.5004e+00  4e+00  6e-16  7e-16
 3: -2.2581e+00 -3.1365e+00  9e-01  4e-16  4e-16
 4: -2.3838e+00 -2.6883e+00  3e-01  2e-16  5e-16
 5: -2.4701e+00 -2.5619e+00  9e-02  2e-16  5e-16
 6: -2.4967e+00 -2.5011e+00  4e-03  2e-16  5e-16
 7: -2.4986e+00 -2.4987e+00  1e-04  2e-16  5e-16
 8: -2.4986e+00 -2.4986e+00  5e-06  5e-16  5e-16
 9: -2.4986e+00 -2.4986e+00  1e-07  5e-16  6e-16
10: -2.4986e+00 -2.4986e+00  1e-09  5e-16  5e-16
11: -2.4986e+00 -2.4986e+00  1e-11  8e-16  6e-16
Optimal solution found.
Número de vectores de soporte: 14
Vectores de soporte:
[[4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [4.8 3.4 1.9 0.2]
 [4.5 2.3 1.3 0.3]
 [6.3 3.3 6.  2.5]
 [4.9 2.5 4.5 1.7]
 [7.7 3.8 6.7 2.2]
 [7.7 2.6 6.9 2.3]
 [6.  2.2 5.  1.5]
 [7.9 3.8 6.4 2. ]
 [6.1 2.6 5.6 1.4]
 [6.9 3.1 5.1 2.3]]


**pcost y dcost**: Estos son el coste primal y dual, respectivamente, en cada iteración del algoritmo. El coste primal se refiere al valor de la función objetivo en el espacio original de variables, mientras que el coste dual es el valor en el espacio de las variables duales. Al principio, estos valores son muy diferentes, lo que indica que la solución aún no es óptima.

**gap**: Esta es la diferencia entre el coste primal y el coste dual. A medida que el algoritmo progresa, este "gap" debe disminuir, indicando que la solución primal y dual se están acercando a la convergencia. Cuando el gap es pequeño, estamos cerca de la solución óptima.

**pres**: La "presión" primal o presión residual primal. Es una medida de cuán bien se están cumpliendo las restricciones del problema primal. Una presión pequeña significa que las restricciones se están satisfaciendo bien.

**dres**: La presión residual dual. Similar a la presión primal, pero para el problema dual. También es una medida de la satisfacción de las restricciones, pero en el espacio dual.

## Algoritmo del Método de Puntos Interiores

### Paso 1: Inicialización

Se selecciona un punto inicial  $x_0$ en el interior del conjunto factible y se establece un parámetro de barrera inicial  μ .

### Paso 2: Solución de Subproblemas

En cada iteración \( k \), se resuelve el siguiente subproblema cuadrático:

\[
\begin{aligned}
\text{minimizar} \quad & f_0(\mathbf{x}) - \mu_k \sum_{i=1}^{m} \log(-f_i(\mathbf{x})),
\end{aligned}
\]

donde $μ_k$ es el valor actual del parámetro de barrera. Este subproblema se resuelve típicamente mediante métodos de Newton.

### Paso 3: Actualización del Parámetro de Barrera

El parámetro de barrera μ se reduce según una estrategia predefinida, como $ \mu_{k+1} = \rho \mu_k $ con $ 0 < \rho < 1 $.

### Paso 4: Condiciones de Terminación

El algoritmo verifica la convergencia basándose en criterios como la norma del gradiente de la función objetivo modificada o la proximidad a las condiciones de Karush-Kuhn-Tucker (KKT).

### Paso 5: Extracción de la Solución

Al converger, se extrae la solución actual como la aproximación óptima.